In [8]:
%matplotlib inline

In [ ]:
# This notebook exports a Catchment_attributes.csv file for Basin Delineation B

In [9]:
import pandas as pds
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pathlib import Path
import geopandas as gpd
import pickle
import datetime as dt
import geopandas as gpds
import geopandas as gpd
import pickle
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
from shapely.geometry import MultiPolygon
from fiona.crs import from_epsg
pds.set_option('display.max_rows', 500)

In [10]:
# Define paths to files
gauges_path = Path(r'C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\gauges_with_splitted_included.shp') 
wsheds_path = Path(r'C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\final_watersheds\Basins_B.shp')

soil_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\soil_attrs_basins_B.csv")
NI_geo_attrs_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\NI_geo_attrs_basins_B.csv")
topo_attrs_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\topo_attrs_Basins_B.csv")
clim_ind_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\Clim_ind_1989_2009.csv")

# Read gauges shapefile
gauges = gpds.read_file(gauges_path)
gauges = gauges.set_index('id')
cols = ['st_numer', 'st_nafn', 'vhm_numer', 'vatnsfall', 'gagnaeigan', 'stada', 'upphaf_rek','dags_aflag', 'elevation']
gauges_reduced = gauges[cols]
gauges_reduced.columns = ['V_no','name','VHM_no','river','data_owner','status','upphaf_rek','dags_aflag', 'elevation']

# Read watershed attributes (calculated in QGIS)
wsheds_attrs = gpds.read_file(wsheds_path)
wsheds_attrs.index = wsheds_attrs['id']
wsheds_attrs = wsheds_attrs.drop(columns=['id'])
wsheds_attrs['lat'] = gauges.geometry.y
wsheds_attrs['lon'] = gauges.geometry.x
# Drop columns from wsheds_attrs that have no values
wsheds_attrs.dropna(axis=1, how='all', inplace=True)
# Extract only the relevant columns from wsheds_attrs
wsheds_attrs = wsheds_attrs[['forest_fra', 'gc_dom', 'strm_dens', 'mvert_dist', 'gc_ss_fra', 'area_calc', 'gc_wb_fra', 'litho_vapy', 'agr_fra', 'gc_py_fra', 'litho_dom', 'gc_ig_fra', 'elon_ratio', 'bare_fra', 'lon', 'litho_pb', 'glac_fra', 'gc_sm_fra', 'lc_dom', 'mvert_ang', 'gc_vb_fra', 'gc_mt_fra', 'litho_vbsr', 'gc_pi_fra', 'lake_fra', 'urban_fra', 'gc_va_fra', 'gc_su_fra', 'litho_pavr', 'litho_vbpy', 'gc_pb_fra', 'litho_vb', 'lat', 'gc_sc_fra', 'gc_pa_fra','lai_max', 'lai_diff', 'ndvi_max', 'ndvi_min','gvf_max', 'gvf_diff','geometry']]

# Read the soil attributes where depth to bedrock is calculated by excluding the glaciated regions
soil_indices = pds.read_csv(soil_path)
soil_indices = soil_indices.set_index('Unnamed: 0')
soil_indices.index = soil_indices.index.rename('id')

NI_geo_attrs = pds.read_csv(NI_geo_attrs_path)
NI_geo_attrs = NI_geo_attrs.set_index('ID')
NI_geo_attrs.index = NI_geo_attrs.index.rename('id')

# Topographical attributes
topo_attrs = pds.read_csv(topo_attrs_path,sep=';')
topo_attrs = topo_attrs.set_index('ID')

# Read ERA-Land data
era5_data = pds.read_csv(clim_ind_path,sep=';') 
era5_data.index=era5_data['ID']
era5_data.index = era5_data.index.rename('id')

In [11]:
print(len(gauges))
print(len(wsheds_attrs))
print(len(era5_data))
print(len(soil_indices))
print(len(NI_geo_attrs))


111
107
107
107
107


In [12]:
# merge all the DataFrames based on their index
merged_dff = wsheds_attrs.sort_index().merge(soil_indices.sort_index(),left_index=True, right_index=True).merge(NI_geo_attrs.sort_index(), left_index=True, right_index=True).merge(era5_data.sort_index(), left_index=True, right_index=True).merge(topo_attrs.sort_index(), left_index=True, right_index=True)

merged_dff.geometry = gauges.geometry
type(merged_dff)

geopandas.geodataframe.GeoDataFrame

In [13]:
attrs_to_export =   ['area_calc',
                     'elev_mean',
                     'elev_med',
                     'elev_ran',
                     'slope_mean',
                     'elev_std',
                     'asp_mean',
                     'gc_dom',
                     'strm_dens',
                     'mvert_dist',
                     'gc_ss_fra',
                     'gc_wb_fra',
                     'litho_vapy',
                     'agr_fra',
                     'gc_py_fra',
                     'litho_dom',
                     'gc_ig_fra',
                     'elon_ratio',
                     'bare_fra',
                     'litho_pb',
                     'glac_fra',
                     'gc_sm_fra',
                     'lc_dom',
                     'mvert_ang',
                     'gc_vb_fra',
                     'gc_mt_fra',
                     'litho_vbsr',
                     'gc_pi_fra',
                     'lake_fra',
                     'forest_fra',
                     'urban_fra',
                     'gc_va_fra',
                     'gc_su_fra',
                     'litho_pavr',
                     'litho_vbpy',
                     'gc_pb_fra',
                     'litho_vb',
                     'gc_sc_fra',
                     'gc_pa_fra',
                     'lai_max',
                     'lai_diff',
                     'ndvi_max',
                     'ndvi_min',
                     'gvf_max',
                     'gvf_diff',
                     'clay_fra',
                     'grav_fra',
                     'soil_tawc',
                     'oc_fra',
                     'silt_fra',
                     'soil_poros',
                     'root_dep',
                     'sand_fra',
                     'bedrk_dep',
                     'g621_fra',
                     'g701_fra',
                     'g743_fra',
                     'g746_fra',
                     'gbinn_fra',
                     'gbnew_fra',
                     'gbold_fra',
                     'ggnew_fra',
                     'ggold_fra',
                     'ghraun_fra',
                     'gmob_fra',
                     'gsgos_fra',
                     'gsinn_fra',
                     'gsn_fra',
                     'gsnew_fra',
                     'gsold_fra',
                     'g_dom_NI',
                     'p_mean',
                     'pet_mean',
                     'aridity',
                     'p_seasonality',
                     'frac_snow',
                     'high_prec_freq',
                     'high_prec_dur',
                     'high_prec_timing',
                     'low_prec_freq',
                     'low_prec_dur',
                     'low_prec_timing']

attributes_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\Catchment_attributes.csv")
merged_dff[attrs_to_export].round(3).to_csv(attributes_path)